This file useses the purchases of the four FAs that we could find (2239-5-LR21, 2239-8-LR23, 2239-4-LR17 and ID 2239-20-LP13 ) and does the following things: 
1. Joins them 
2. merges them with the car data
3. estimates a demand model

In [244]:
import os
import pandas as pd
from pandasgui import show
import socket
import numpy as np
from unidecode import unidecode
import unicodedata
from difflib import SequenceMatcher
from fuzzywuzzy import process, fuzz


### Functions

In [245]:
def clean_text(text):
    if isinstance(text, str):
        return unidecode(text)
    return text


def fix_encoding(text):
    if isinstance(text, str):
        replacements = {
            'Ã³': 'ó',
            'Ã­': 'í',
            'Ã¡': 'á',
            'Ã©': 'é',
            'Ã±': 'ñ',
            'Ãº': 'ú',
            'Ã¼': 'ü',
            'Ã': 'í'
        }
        for wrong, right in replacements.items():
            text = text.replace(wrong, right)
    return text

def process_prices(df):
    """
    Process prices in a dataframe according to specific rules using vectorized operations:
    1. Process all prices in scientific notation first
    2. For remaining prices with commas, remove everything after the comma
    3. For USD prices, multiply by 700
    
    Parameters:
    df (pd.DataFrame): Input dataframe with Moneda and Precio Unitario columns
    
    Returns:
    pd.DataFrame: Processed dataframe
    pd.DataFrame: Rows with processing issues
    """
    # Create a copy to avoid modifying the original
    df_processed = df.copy()
    
    # Ensure the required columns exist
    required_cols = ['Moneda', 'Precio Unitario']
    if not all(col in df_processed.columns for col in required_cols):
        raise ValueError(f"Missing required columns. Need: {required_cols}")
    
    # Convert Precio Unitario to string type for string operations
    df_processed['Precio Unitario'] = df_processed['Precio Unitario'].astype(str)
    
    # Create masks for different conditions
    has_scientific_notation = df_processed['Precio Unitario'].str.contains('e', case=False, na=False)
    has_comma_mask = df_processed['Precio Unitario'].str.contains(',', na=False)
    is_usd_mask = df_processed['Moneda'] == 'USD'
    
    # 1. Process scientific notation first
    scientific_values = df_processed[has_scientific_notation]['Precio Unitario']
    if not scientific_values.empty:
        # Replace comma with dot for proper float conversion
        scientific_values = scientific_values.str.replace(',', '.')
        df_processed.loc[has_scientific_notation, 'Precio Unitario'] = (
            pd.to_numeric(scientific_values, errors='coerce')
        )
    
    # 2. Process remaining prices with commas (excluding already processed scientific notation)
    remaining_comma_mask = has_comma_mask & ~has_scientific_notation
    if any(remaining_comma_mask):
        df_processed.loc[remaining_comma_mask, 'Precio Unitario'] = (
            df_processed.loc[remaining_comma_mask, 'Precio Unitario']
            .str.split(',')
            .str[0]
        )
    
    # Convert all prices to numeric
    df_processed['Precio Unitario'] = pd.to_numeric(df_processed['Precio Unitario'], errors='coerce')
    
    # 3. Multiply USD prices by 700
    df_processed.loc[is_usd_mask, 'Precio Unitario'] = df_processed.loc[is_usd_mask, 'Precio Unitario'] * 700
    
    # Find problematic rows (where conversion to numeric failed)
    problematic_rows = df_processed[df_processed['Precio Unitario'].isna()].copy()
    print(f"Number of problematic rows: {len(problematic_rows)}")

    return df_processed


## Read the purchases for the FAs and join them

In 2017 we have also leases and repairs of cars. This has to be fixed. 

In [246]:
transac_cm_2013_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transacciones_cm_2013.csv')
transac_cm_2013_df = pd.read_csv(transac_cm_2013_path)
print(f"Dimensions of transac_cm_2013_df: {transac_cm_2013_df.shape}")
transac_cm_2013_df.head()

Dimensions of transac_cm_2013_df: (0, 1)


,Unnamed: 0


### 2017

In [247]:
transac_cm_2017_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transacciones_cm_2017.csv')
transac_cm_2017_df = pd.read_csv(transac_cm_2017_path)
transac_cm_2017_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
print(f"Dimensions of transac_cm_2017_df: {transac_cm_2017_df.shape}")

# Extract the year from the 'Fecha Envío OC' column
transac_cm_2017_df['year'] = pd.to_datetime(transac_cm_2017_df['Fecha Envío OC']).dt.year

# Count the number of observations for each year
year_counts = transac_cm_2017_df['year'].value_counts().sort_index()
print(year_counts)

nro_counts = transac_cm_2017_df['Nro Licitación Pública'].value_counts().sort_index() 
print(nro_counts)

transac_cm_2017_df.head()
transac_cm_2017_df = process_prices(transac_cm_2017_df)

Dimensions of transac_cm_2017_df: (19997, 45)
year
2018    3574
2019    6056
2020    5174
2021    5193
Name: count, dtype: int64
Nro Licitación Pública
2239-4-LR17    19997
Name: count, dtype: int64
Number of problematic rows: 0


clean the 2017 data

In [248]:
#clean based on the 'Convenio Marco' column
convenio_aux = transac_cm_2017_df['Convenio Marco'].unique()
#print(convenio_aux)
transac_cm_2017_df = transac_cm_2017_df[transac_cm_2017_df['Convenio Marco'] == 'Compra de Vehículos Livianos y Medianos']


#clean based on the 'Tipo de Producto' column (drop Productos with less than 30 observations, they were weird products like 'City car' or 'oficinas moviles')
counts = transac_cm_2017_df['Tipo de Producto'].value_counts()
print(counts)
to_drop = counts[counts < 30].index
transac_cm_2017_df = transac_cm_2017_df[
    ~transac_cm_2017_df['Tipo de Producto'].isin(to_drop)
]
transac_cm_2017_df = transac_cm_2017_df[transac_cm_2017_df['Tipo de Producto'] != 'MANTENCIÓN PREVENTIVA VEHÍCULO LIVIANO Y MEDIANO']



Tipo de Producto
MANTENCIÓN PREVENTIVA VEHÍCULO LIVIANO Y MEDIANO    14843
CAMIONETA                                            1178
SUV                                                   554
MINIBUS                                               349
SEDÁN                                                 234
AMBULANCIAS                                           184
VEHÍCULOS POLICIALES                                  119
CAMIÓN LIVIANO                                        103
FURGÓN                                                 87
HATCHBACK                                              41
CITY CAR                                               29
OFICINAS MOVILES                                        4
VEHÍCULO TRASLADO MENORES                               3
VEHÍCULO TRASLADO INTERNOS                              2
Name: count, dtype: int64


In [249]:
# check that the data makes sense and is clean
counts = transac_cm_2017_df['Tipo de Producto'].value_counts()
print(counts)
summary_precio_unitario = transac_cm_2017_df['Precio Unitario'].describe()
print(summary_precio_unitario)


Tipo de Producto
CAMIONETA               1178
SUV                      554
MINIBUS                  349
SEDÁN                    234
AMBULANCIAS              184
VEHÍCULOS POLICIALES     119
CAMIÓN LIVIANO           103
FURGÓN                    87
HATCHBACK                 41
Name: count, dtype: int64
count    2.849000e+03
mean     1.761404e+07
std      1.023039e+07
min      4.058600e+06
25%      1.269076e+07
50%      1.535040e+07
75%      1.899000e+07
max      1.240800e+08
Name: Precio Unitario, dtype: float64


### 2021

In [250]:
transac_cm_2021_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones', 'transacciones_cm_2021.csv')
transac_cm_2021_df = pd.read_csv(transac_cm_2021_path)
transac_cm_2021_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
print(f"Dimensions of transac_cm_2021_df: {transac_cm_2021_df.shape}")

 # Fix weird characters in columns and in text 
#transac_cm_2021_df.columns = [clean_text(col) for col in transac_cm_2021_df.columns]
#for column in transac_cm_2021_df.select_dtypes(include=['object']).columns:
#    transac_cm_2021_df[column] = transac_cm_2021_df[column].apply(clean_text)



transac_cm_2021_df.columns = [fix_encoding(col) for col in transac_cm_2021_df.columns]
for column in transac_cm_2021_df.select_dtypes(include=['object']).columns:
    transac_cm_2021_df[column] = transac_cm_2021_df[column].apply(fix_encoding)


transac_cm_2021_df['year'] = pd.to_datetime(transac_cm_2021_df['Fecha Envío OC']).dt.year


# Count the number of observations for each year
year_counts = transac_cm_2021_df['year'].value_counts().sort_index()
print(year_counts)

nro_counts = transac_cm_2021_df['Nro Licitación Pública'].value_counts().sort_index() 

print(nro_counts)

transac_cm_2021_df = process_prices(transac_cm_2021_df)
transac_cm_2021_df.head()

Dimensions of transac_cm_2021_df: (1071, 45)
year
2021      7
2022    277
2023    721
2024     66
Name: count, dtype: int64
Nro Licitación Pública
2239-5-LR21    1071
Name: count, dtype: int64
Number of problematic rows: 0


,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,Especificación del Comprador,...,Nombre Empresa,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor,year
0,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,2564-270-CM21,Orden de Compra: 2564-270-CM21,2021-10-20,Enviada a Proveedor,No,NaN,(1840093) CAMIONETAS MAXUS T60 T60 DX 4X4 MAC...,...,Mediterraneo Automotores S.A.,NaN,NaN,Orden de Compra codigo: 2564-270-CM21 dirigida...,30 días contra la recepción conforme de la fac...,3555.0,92414.0,824156,27979,2021
1,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,4360-75-CM21,Camioneta maxus d/c 4x4,2021-12-24,Enviada a Proveedor,No,NaN,(1840095) CAMIONETAS MAXUS T60 T60 DX 4X4 MAC...,...,Mediterraneo Automotores S.A.,NaN,NaN,Orden de Compra codigo: 4360-75-CM21 dirigida ...,30 días contra la recepción conforme de la fac...,5277.0,135561.0,824156,27979,2021
2,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,3140-819-CM21,"Orden de Compra: 3140-819-CM21, ADQUISICIíN D...",2021-12-21,Enviada a Proveedor,No,NaN,(1848052) CAMIONETA NISSAN NAVARA DC SE 2.3D M...,...,SALINAS Y FABRES SOCIEDAD ANONIMA,NaN,NaN,Orden de Compra codigo: 3140-819-CM21 dirigida...,30 días contra la recepción conforme de la fac...,4061.0,113735.0,26569,26778,2021
3,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,1084170-110-CM21,Camioneta para Uso del SLEP,2021-12-28,Recepción Conforme,No,NaN,(1840060) CAMIONETAS MAHINDRA PIK UP PICK UP R...,...,BRUNO FRITSCH S.A.,NaN,NaN,1.\tAceptar la OC y facturar después de entreg...,30 días contra la recepción conforme de la fac...,1084170.0,1675231.0,24507,24823,2021
4,2239-5-LR21,5800296.0,CM Adquisición de Vehículos SUV y Camionetas (...,4236-722-CM21,Orden de Compra: 4236-722-CM21,2021-12-29,Aceptada,No,NaN,(1848008) CAMIONETA TOYOTA HILUX 4X4 2.4 MT DX...,...,Maritano y Ebensperger Ltda.,NaN,NaN,Orden de Compra codigo: 4236-722-CM21 dirigida...,30 días contra la recepción conforme de la fac...,5154.0,127036.0,62896,61164,2021


clean the 2021 data

In [251]:
#not necessary to do anything with Convenio Marco, since there is no subletting in 2021
convenio_aux = transac_cm_2021_df['Convenio Marco'].unique()
#print(convenio_aux)

# not necessary to do anything with Tipo de Producto since there are only SUVs and pick-ups
counts = transac_cm_2021_df['Tipo de Producto'].value_counts()
print(counts)

# check that the data makes sense and is clean
counts = transac_cm_2021_df['Tipo de Producto'].value_counts()
print(counts)
summary_precio_unitario = transac_cm_2021_df['Precio Unitario'].describe()
print(summary_precio_unitario)

Tipo de Producto
CAMIONETA    800
SUV          271
Name: count, dtype: int64
Tipo de Producto
CAMIONETA    800
SUV          271
Name: count, dtype: int64
count    1.071000e+03
mean     1.965587e+07
std      5.451048e+06
min      7.651600e+06
25%      1.579042e+07
50%      1.842240e+07
75%      2.419873e+07
max      3.818530e+07
Name: Precio Unitario, dtype: float64


### merge 2021 data with auctions data

The merging criteria should be 
1. Marca(transactions) coincides with Marca(auction)
2. Macrozona(transactions, created) coincides with Macrozona(auction), 
3. Tipo de Producto (transactions) coincides with Categoria (auction)
4. Rut proveedor (transactions) coincides with Rut(auction) 

In [252]:
#1. create a merge variable in transactions data 
transac_cm_2021_df['Macrozona'] = transac_cm_2021_df['Producto'].str.extract(r'MACROZONA\s+(\w+)')
transac_cm_2021_df['exact_merge_var'] = (transac_cm_2021_df['Marca'].astype(str) + ' '
                                        + transac_cm_2021_df['Macrozona'].astype(str) + ' '
                                        + transac_cm_2021_df['Tipo de Producto'].astype(str) + ' '
                                        + transac_cm_2021_df['Rut Proveedor'].astype(str))

transac_cm_2021_df['exact_merge_var'] = transac_cm_2021_df['exact_merge_var'].str.upper()

#show(transac_cm_2021_df) 

In [253]:
#2. import the acution data and create a merge variable
auction_path = r"C:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\interm_data\CM_auctions"
auction_df = pd.read_excel(os.path.join(auction_path, 'auction21.xlsx'))


#2.1 create the matching variable 
print(auction_df.columns)
auction_df.loc[auction_df['Categoría_x'] == 'Camionetas', 'Categoría_x'] = 'CAMIONETA'
auction_df['exact_merge_var'] = (auction_df['Marca_x'].astype(str) + ' '
                                        + auction_df['Macrozona_x'].astype(str) + ' '
                                        + auction_df['Categoría_x'].astype(str) + ' '
                                        + auction_df['RUT_x'].astype(str))
auction_df['exact_merge_var'] = auction_df['exact_merge_var'].str.upper()

#2.2 keep only observations that were selected in the auction stage. 
print(auction_df['Estado Final'].unique())
auction_df = auction_df[auction_df['Estado Final'] == 'ADJUDICA']


#auction_df.sort_values(by='Marca', inplace=True)
auction_df.sort_values(by='exact_merge_var', inplace=True)
#show(auction_df)

Index(['RUT_x', 'Nombre_x', 'Estado Anterior_x', 'Categoría_x', 'Vehículo',
       'Macrozona_x', 'Gama_x', 'Pgm', 'Puntaje Precio_x', 'PUNTAJE OFERTA',
       'Estado Final', 'Marca_x', 'Precio', 'Modelo', 'Versión',
       'Precio + Despacho', 'Precio Macrozona-Gama', 'Puntaje Precio_y'],
      dtype='object')
['INADMISIBLE' 'ADJUDICA' 'No Adjudica']


In [254]:
# 3. merge the two datasets
transac_cm_2021_df['Index'] = transac_cm_2021_df.index + 1

# Merge on 'exact_merge_var'
merged_df = transac_cm_2021_df.merge(auction_df, on='exact_merge_var', how='left', indicator=True)

# 1. Calculate the share of observations in auction_21 that are merged
share_merged = merged_df['_merge'].value_counts(normalize=True).get('both', 0)
print(f"Share of observations in auction_21 that are merged: {share_merged:.2%}")

# 2. Create a variable that counts how many matches are found in the other file
match_counts = auction_df.groupby('exact_merge_var').size()
transac_cm_2021_df['match_count'] = transac_cm_2021_df['exact_merge_var'].map(match_counts).fillna(0).astype(int)

#unmatched_df = merged_df[merged_df['_merge'] == 'left_only']
#print(unmatched_df.shape)

#show(merged_df)

Share of observations in auction_21 that are merged: 32.02%


In [255]:
# 4. keep only one obs in cases with various matches 

#4.1 create the variables to compute similarity 
#print(merged_df.columns)
merged_df['Vehículo_y'] = merged_df['Marca'].astype(str) + ' '  + merged_df['Modelo_x'].astype(str)
merged_df['Vehículo_y'] = merged_df['Vehículo_y'].str.upper()
merged_df['Vehículo'] = merged_df['Vehículo'].str.upper()


#4.2 create the similarity measure 
merged_df['Vehículo'] = merged_df['Vehículo'].fillna('').astype(str).str.upper()
merged_df['Vehículo_y'] = merged_df['Vehículo_y'].fillna('').astype(str).str.upper()
def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()
merged_df['similarity'] = merged_df.apply(
    lambda row: calculate_similarity(row['Vehículo'], row['Vehículo_y']), axis=1
)

# 4.3. For observations where match_count_x > 1, keep only the observation with the highest similarity
merged_df = merged_df.loc[
    merged_df.groupby('Index')['similarity'].idxmax()
]
#show(merged_df)

# 5. rename the merged_df 
transac_cm_2021_df = merged_df.copy()
transac_cm_2021_df['Modelo'] = transac_cm_2021_df['Modelo_x']
#show(transac_cm_2021_df)

### 2023

import 2023 data

In [256]:
transac_cm_2023_path = os.path.join('..', 'interm_data',  'yearly_data', 'Transacciones', 'transacciones_cm_2023.csv')
transac_cm_2023_df = pd.read_csv(transac_cm_2023_path)
transac_cm_2023_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
print(f"Dimensions of transac_cm_2023_df: {transac_cm_2023_df.shape}")

transac_cm_2023_df.columns = [fix_encoding(col) for col in transac_cm_2023_df.columns]
for column in transac_cm_2023_df.select_dtypes(include=['object']).columns:
    transac_cm_2023_df[column] = transac_cm_2023_df[column].apply(fix_encoding)


transac_cm_2023_df['year'] = pd.to_datetime(transac_cm_2023_df['Fecha Envío OC']).dt.year


# Count the number of observations for each year
year_counts = transac_cm_2023_df['year'].value_counts().sort_index()
print(year_counts)

nro_counts = transac_cm_2023_df['Nro Licitación Pública'].value_counts().sort_index() 
print(nro_counts)

transac_cm_2023_df = process_prices(transac_cm_2023_df)


transac_cm_2023_df.head()

Dimensions of transac_cm_2023_df: (366, 45)
year
2024    366
Name: count, dtype: int64
Nro Licitación Pública
2239-8-LR23    366
Name: count, dtype: int64
Number of problematic rows: 0


,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,Especificación del Comprador,...,Nombre Empresa,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor,year
0,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,4062-187-CM24,Orden de Compra: 4062-187-CM24,2024-03-12,Recepcion Conforme,No,NaN,(2042525) CAMIONETA MAXUS T60 E6 GL 4X4 MT 202...,...,AUTOMOTRIZ SERVIMAQ SPA,NaN,NaN,Orden de Compra codigo: 4062-187-CM24 dirigida...,30 dias contra la recepcion conforme de la fac...,4982,120001,100329,97856,2024
1,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,3577-41-CM24,Orden de Compra: 3577-41-CM24,2024-03-14,Aceptada,No,NaN,(2041399) CAMIONETA TOYOTA HILUX DX 2.4 4X4 MT...,...,BRUNO FRITSCH S.A.,NaN,NaN,Orden de Compra codigo: 3577-41-CM24 dirigida ...,30 dias contra la recepcion conforme de la fac...,4497,115285,24507,24823,2024
2,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,3030-177-CM24,CAMIONETA JMC PARA DEPTO. DE SALUD,2024-03-14,Aceptada,No,NaN,(2041534) CAMIONETA JMC VIGUS NEW WORK 4X4 MT ...,...,AUTOMOTRIZ CORDILLERA S.A.,NaN,NaN,CAMIONETA JMC VIGUS NEW WORK 4X4 MT 2024 PARA ...,30 dias contra la recepcion conforme de la fac...,3978,113252,32639,32414,2024
3,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,2381-308-CM24,ADQUISICIíN DE 01 CAMIONETA TODO TERRENO PARA...,2024-03-18,Aceptada,No,NaN,(2041392) CAMIONETA TOYOTA HILUX DX 2.4 4X4 MT...,...,BRUNO FRITSCH S.A.,NaN,NaN,ADQUISICIíN DE 01 CAMIONETA TODO TERRENO PARA...,30 dias contra la recepcion conforme de la fac...,3373,86872,24507,24823,2024
4,2239-8-LR23,5802342.0,Convenio Marco de Adquisición de Vehículos,316-212-CM24,Orden de Compra: 316-212-CM24,2024-03-20,Aceptada,No,NaN,(2041566) CAMIONETA JMC VIGUS NEW PLUS 4X4 MT ...,...,AUTOMOTRIZ CORDILLERA S.A.,NaN,NaN,Orden de Compra codigo: 316-212-CM24 dirigida ...,30 dias contra la recepcion conforme de la fac...,1313,7233,32639,32414,2024


clean the 2023 data

In [257]:
#not necessary to do anything with Convenio Marco, since there is no subletting in 2023
convenio_aux = transac_cm_2023_df['Convenio Marco'].unique()
#print(convenio_aux)

# clean based on the 'Tipo de Producto' column
counts = transac_cm_2023_df['Tipo de Producto'].value_counts()
#print(counts)
to_drop = ['RETROEXCAVADORA','BUS PESADO' ]
transac_cm_2023_df = transac_cm_2023_df[
    ~transac_cm_2023_df['Tipo de Producto'].isin(to_drop)
]


# Normalize text by removing accents
transac_cm_2023_df['Tipo de Producto'] = transac_cm_2023_df['Tipo de Producto'].apply(
    lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode('ASCII')
)

# Step 1: Create a mapping dictionary for all variations
replacements = {
    'SEDiN': 'SEDÁN',
    'FURGiN' : 'FURGÓN'
}

# Step 2: Replace all variations using the mapping
transac_cm_2023_df['Tipo de Producto'] = transac_cm_2023_df['Tipo de Producto'].replace(replacements)

print( transac_cm_2023_df['Tipo de Producto'].value_counts())



# check that the data makes sense and is clean
counts = transac_cm_2023_df['Tipo de Producto'].value_counts()
print(counts)
summary_precio_unitario = transac_cm_2023_df['Precio Unitario'].describe()
print(summary_precio_unitario)



Tipo de Producto
CAMIONETA    211
SUV           83
MINIBUS       31
SEDÁN         21
FURGÓN         8
CARGO          8
Name: count, dtype: int64
Tipo de Producto
CAMIONETA    211
SUV           83
MINIBUS       31
SEDÁN         21
FURGÓN         8
CARGO          8
Name: count, dtype: int64
count    3.620000e+02
mean     2.091208e+07
std      6.801193e+06
min      6.844800e+06
25%      1.642131e+07
50%      2.087100e+07
75%      2.573550e+07
max      5.243080e+07
Name: Precio Unitario, dtype: float64


### Join the purchases from all the different FAs

In [258]:
# Concatenate the dataframes vertically
transac = pd.concat([transac_cm_2017_df, transac_cm_2021_df, transac_cm_2023_df], axis=0)

# Reset the index if needed
transac_df = transac.reset_index(drop=True)
print(transac_df.columns)
print(f"Dimensions of transac_df: {transac_df.shape}")
transac_df.head(2)


output_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones')
transac_df.to_csv(os.path.join(output_path, 'joined_transac.csv'), index=False)

Index(['Nro Licitación Pública', 'Id Convenio Marco', 'Convenio Marco',
       'CodigoOC', 'NombreOC', 'Fecha Envío OC', 'EstadoOC',
       'Proviene de Gran Compra', 'idGranCompra',
       'Especificación del Comprador', 'IDProductoCM', 'Producto',
       'Nombre Producto ONU', 'Tipo de Producto', 'Marca', 'Modelo',
       'Precio Unitario', 'Cantidad', 'TotaLínea(Neto)', 'Moneda',
       'Monto Total OC Neto', 'Descuento Global OC', 'Cargos Adicionales OC',
       'Subtotal OC', 'Impuestos', 'Monto Total OC', 'Rut Unidad de Compra',
       'Unidad de Compra', 'Razón Social Comprador', 'Dirección Unidad Compra',
       'Comuna Unidad Compra', 'Región Unidad de Compra', 'Institución',
       'Sector', 'Rut Proveedor', 'Nombre Proveedor Sucursal',
       'Nombre Empresa', 'Comuna del Proveedor', 'Región del Proveedor',
       'Observaciones', 'Forma de Pago', 'Orgcode_Comprador',
       'Entcode_Comprador', 'Orgcode_Proveedor', 'Entcode_Proveedor', 'year',
       'Modelo_x', 'Macrozona'

## Data cleaning

### Remove the observations that are not purchases of cars, but rather maintenance or leases. 

Do some cleaning based on Tipo de Producto (this is harmless, already did it for each file)

In [259]:
#%reset -f
output_path = os.path.join('..', 'interm_data', 'yearly_data', 'Transacciones')


In [260]:
df = pd.read_csv(os.path.join(output_path, 'joined_transac.csv'))

print(f"Shape of transac_df: {df.shape}")
tipo_producto_values = df['Tipo de Producto'].unique()
print(tipo_producto_values)

# Create a list of product types to drop
drop_product_types = ['ARRIENDO', 'MANTENCIÓN', 'OFICINAS', 'RODILLO',
                      'PESADO', 'BULL', 'TRASLADO', 'EXCAVA', 'TRACTO', 'ADORA', 'TOLVA', 'CARGADOR', 
                      'VEHÍCULOS POLICIALES', 'AMBULANCIAS']

# Create a regex pattern to match any of the product types in the list
pattern = '|'.join(drop_product_types)

# Drop observations where 'Tipo de Producto' contains an element of the list
drop_mask = ~df['Tipo de Producto'].str.contains(pattern, case=False, na=False)
df = df[drop_mask]
print(f"Shape of transac_df after dropping specified product types: {df.shape}")

Shape of transac_df: (4282, 71)
['MINIBUS' 'SUV' 'CAMIONETA' 'SEDÁN' 'FURGÓN' 'HATCHBACK' 'CAMIÓN LIVIANO'
 'VEHÍCULOS POLICIALES' 'AMBULANCIAS' 'CARGO']
Shape of transac_df after dropping specified product types: (3979, 71)


Do some cleaning based on 'Convenio Marco'  (this is harmless, already did it for each file)

In [261]:
print(f"Shape of transac_df: {df.shape}")
tipo_convenio = df['Convenio Marco'].unique()
print(tipo_convenio)

avg_precio = df.groupby('Convenio Marco')['Precio Unitario'].mean()
print(avg_precio)


# Remove products from FA of heavy and medium heavy vehicles in the Convenio Marco column
heavy_mask = ~df['Convenio Marco'].str.contains('Compra de Vehículos Pesados y Maquinarias', na=False)
df = df[heavy_mask]
print(f"Shape of transac_df: {df.shape}")


Shape of transac_df: (3979, 71)
['Compra de Vehículos Livianos y Medianos'
 'CM Adquisición de Vehículos SUV y Camionetas (Magento)'
 'CM Adquisición de Vehículos SUV y Camionetas'
 'Convenio Marco de  Adquisición de Vehículos']
Convenio Marco
CM Adquisición de Vehículos SUV y Camionetas              2.001232e+07
CM Adquisición de Vehículos SUV y Camionetas (Magento)    1.683101e+07
Compra de Vehículos Livianos y Medianos                   1.487118e+07
Convenio Marco de  Adquisición de Vehículos               2.091208e+07
Name: Precio Unitario, dtype: float64
Shape of transac_df: (3979, 71)


In [262]:
tipo_producto_counts = df['Convenio Marco'].value_counts()
print(tipo_producto_counts)

Convenio Marco
Compra de Vehículos Livianos y Medianos                   2546
CM Adquisición de Vehículos SUV y Camionetas               951
Convenio Marco de  Adquisición de Vehículos                362
CM Adquisición de Vehículos SUV y Camionetas (Magento)     120
Name: count, dtype: int64


In [263]:
#share of 2021 obs for whicch we have auction data
grouped = df.groupby(['Nro Licitación Pública', '_merge']).size().reset_index(name='count')
grouped['total'] = grouped.groupby('Nro Licitación Pública')['count'].transform('sum')
grouped['share'] = grouped['count'] / grouped['total']
print(grouped)

  Nro Licitación Pública     _merge  count  total     share
0            2239-5-LR21       both    296   1071  0.276377
1            2239-5-LR21  left_only    775   1071  0.723623


In [264]:
print(df.value_counts('Nro Licitación Pública'))

Nro Licitación Pública
2239-4-LR17    2546
2239-5-LR21    1071
2239-8-LR23     362
Name: count, dtype: int64


## Pair the variables of the model with the variables of the data     
        
    'IDProductoCM' -> j 
    To create product characteristics: 'Tipo de Producto', 'Marca', 'Nombre Producto ONU'
    'Precio Unitario'-> net not including taxes, just changes the scale of the price parameter 
    'Rut Unidad de Compra' -> i 
    To create the groups (k):  'Región Unidad de Compra', 'Sector'
     'year' -> t 

     'Modelo'-> to do the match with the product characteristics. 
        
Variables not directly paired but which could be useful: 
    'Nro Licitación Pública', 'Id Convenio Marco', 'Convenio Marco', 'CodigoOC', 'Fecha Envío OC', 'Cantidad', 'Rut Proveedor', 
    'Nombre Proveedor Sucursal', 'Orgcode_Comprador', 'Entcode_Comprador', 


In [265]:
# Variables to keep - directly paired with model
model_vars = ['IDProductoCM', 'Modelo',  'Tipo de Producto', 'Marca', 'Nombre Producto ONU',
              'Precio Unitario', 'Rut Unidad de Compra', 'Región Unidad de Compra',
              'Sector', 'year']

# Additional useful variables
extra_vars = ['Nro Licitación Pública', 'Convenio Marco', 'CodigoOC',
              'Fecha Envío OC', 'Cantidad', 'Rut Proveedor',
              'Nombre Proveedor Sucursal']

#extra_vars2 = list(auction_df.columns)
#elements_to_remove = ['RUT_x', 'Nombre_x', 'Estado Anterior_x' ]
#extra_vars2 = [x for x in extra_vars2 if x not in elements_to_remove]
extra_vars2 = [ 'Vehículo', 'Macrozona_x', 'Gama_x', 'Pgm', 'Puntaje Precio_x', 'PUNTAJE OFERTA', 
               'Marca_x', 'Precio', 'Modelo', 'Versión', 'Precio + Despacho', 'Precio Macrozona-Gama', 'Puntaje Precio_y', 'exact_merge_var']

# Create new dataframe with only the specified columns
df = df[ model_vars + extra_vars + extra_vars2]

print(f"Dimensions of transac_df: {df.shape}")


Dimensions of transac_df: (3979, 31)


store the data

In [266]:
df.to_csv(os.path.join(output_path, 'joined_cleaned_transac.csv'), index=False)

In [267]:
print(df.value_counts('year'))

year
2019    898
2018    767
2023    721
2020    504
2024    428
2021    384
2022    277
Name: count, dtype: int64


### Answer the following questions 
- are firms selling multiple products? Yes
- What is the share of firms for which we have the auction data? 28%
- what shar of firms have a binding price ceiling? 


In [268]:
# Group by supplier and count unique products
supplier_products = df.groupby('Nombre Proveedor Sucursal')['IDProductoCM'].nunique().sort_values(ascending=False)
print(supplier_products)

n_unique_products = df['IDProductoCM'].nunique()
print(f"Total number of unique products: {n_unique_products}")

Nombre Proveedor Sucursal
Salinas y Fabres S.A.                            296
SALAZAR ISRAEL                                   204
Automotores Gildemeister CAMIONES                185
Dercosa                                          143
BRUNO FRITSCH S.A.                               113
Maritano y Ebensperger Ltda.                      92
Mediterraneo Automotores SA                       85
PORTILLO S A - Casa Matriz                        59
Kovacs SpA Seminario 385 Providencia              46
COMERCIAL ANFRUNS Y COMPAíIA SPA                 46
POMPEYO CARRASCO Y CIA. LTDA.                     45
Anfruns Motors SPA                                41
AUTOMOTORA MELHUISH SPA                           38
Comercial Motores de los Andes SpA                36
Curifor Coquimbo                                  34
Derco SPA                                         33
DIFOR CHILE SOCIEDAD ANONIMA                      31
Automotriz Tecnosur Limitada                      29
PORSCHE CHILE SPA   

In [269]:
print(df.columns)
price_list = [ 'Precio Unitario', 'Pgm', 'Puntaje Precio_x', 'Precio', 'Precio + Despacho', 'Precio Macrozona-Gama']

df_filtered = df[df['Nro Licitación Pública'] == '2239-5-LR21']

df_filtered = df_filtered[price_list]

Index(['IDProductoCM', 'Modelo', 'Tipo de Producto', 'Marca',
       'Nombre Producto ONU', 'Precio Unitario', 'Rut Unidad de Compra',
       'Región Unidad de Compra', 'Sector', 'year', 'Nro Licitación Pública',
       'Convenio Marco', 'CodigoOC', 'Fecha Envío OC', 'Cantidad',
       'Rut Proveedor', 'Nombre Proveedor Sucursal', 'Vehículo', 'Macrozona_x',
       'Gama_x', 'Pgm', 'Puntaje Precio_x', 'PUNTAJE OFERTA', 'Marca_x',
       'Precio', 'Modelo', 'Versión', 'Precio + Despacho',
       'Precio Macrozona-Gama', 'Puntaje Precio_y', 'exact_merge_var'],
      dtype='object')


In [270]:
# Only calculate for observations where neither 'Precio Unitario' nor 'Precio' is missing
df_filtered.loc[df_filtered['Precio Unitario'].notna() & df_filtered['Precio'].notna(), 'binding_ceiling'] = (
    df_filtered['Precio Unitario'] > 1.1*df_filtered['Precio']
).astype(int)
shares = df_filtered['binding_ceiling'].value_counts(normalize=True, dropna=True) * 100

print("Shares for 'binding_ceiling':", shares)

 
# Only calculate for observations where neither 'Precio Unitario' nor 'Precio' is missing
df_filtered['Precio + Despacho'] = pd.to_numeric(df_filtered['Precio + Despacho'], errors='coerce')
df_filtered.loc[df_filtered['Precio Unitario'].notna() & df_filtered['Precio'].notna(), 'binding_ceiling2'] = (
    df_filtered['Precio Unitario'] > 1.1*df_filtered['Precio + Despacho']
).astype(int)
shares = df_filtered['binding_ceiling2'].value_counts(normalize=True, dropna=True) * 100

print("Shares for 'binding_ceiling':", shares)

Shares for 'binding_ceiling': binding_ceiling
1.0    96.283784
0.0     3.716216
Name: proportion, dtype: float64
Shares for 'binding_ceiling': binding_ceiling2
1.0    95.27027
0.0     4.72973
Name: proportion, dtype: float64


In [271]:
# 1. Share of observations where 'Precio Unitario' is greater than 'Precio'
share_precio_unitario_greater = (df_filtered['Precio Unitario'] > df_filtered['Precio']).mean()
share_precio_greater = (df_filtered['Precio Unitario'] <= df_filtered['Precio']).mean()

print(f"Share where 'Precio Unitario' > 'Precio': {share_precio_unitario_greater:.2%}")
print(f"Share where 'Precio' >= 'Precio Unitario': {share_precio_greater:.2%}")

# 2. Average difference when 'Precio Unitario' is less than 'Precio'
average_difference = (df_filtered['Precio'] - df_filtered['Precio Unitario']).mean()

print(f"On average, 'Precio Unitario' is {average_difference:.2f} lower than 'Precio'")
#show(df_filtered)


Share where 'Precio Unitario' > 'Precio': 27.17%
Share where 'Precio' >= 'Precio Unitario': 0.47%
On average, 'Precio Unitario' is -6151351.29 lower than 'Precio'


## Merge product characteristics (still to do, Pablo has to send me the code)

1. take transac_df and obtain a dataset with model-id where model is created in the same way pablo created model to do the web-scraping 
2. do the merge with the web-scraped data of pablo to obtain a dataset with model-id-prod_char 
3. decrease number of models (e.g rav4 and rav4 mt are the same car) to have dataset  model-id-prod_char-id2 where id2 is a more coarse measure of id. 
4. merge with purchases according to id
5. estimate demand based on id2

### 1. take transac_df and obtain a dataset with model-id where model is created in the same way pablo created model to do the web-scraping 

In [ ]:
df = pd.read_csv(os.path.join(output_path, 'joined_cleaned_transac.csv'))

# Get value counts as a Series
modelos = df.value_counts('Modelo')

# Iterate through the key-value pairs using .iteritems()
for modelo, count in modelos.items():
    print(f'{modelo}: {count}')

H-1 TQ MB 2.5 CRDI MT 12S FL: 144
NP300 DC SE 2.3D MT 4WD: 92
HILUX 2.4L DX CD 4X4: 69
VIGUS WORK LX 4X4 DIESEL G4: 59
HILUX 2.4 4X4 MT DX: 57
T60 2.8L DX 4X4: 55
T60 DX 4X4: 53
HILUX 4X4 2.4 MT DX: 53
T60 DX 4X2: 35
TUCSON NX4 2.0 AT 4WD PLUS: 33
NEW RAV4 LE 6MT AWD: 32
VIGUS DOBLE CABINA 4X2 DIESEL: 32
NP300 DC SE 2.3D MT: 31
TUCSON NX4 2.0 MT PLUS: 29
HILUX 2.4 4X4 MT SR: 29
H-1  MB 2.5 CRDI 6M/T GLS 10S AC 2AB ABS: 28
NEW DMAX CC 2.5TD 4WD: 27
L200 4X4 KATANA CRM: 27
RAV4 LUJO 2.0 4X4 MT: 26
COLORADO LT MT 2.8TD 4WD M: 26
HILUX 4X4 2.4 MT DX DIESEL: 25
HILUX DX 2.4 4X4 MT 2024: 25
HILUX 2.4 4X2 MT DX (WIDE): 25
T60 4X4 GL MT E6: 24
NAVARA 2.3D DC SE 4X4: 24
HILUX 2.4L DX CD 4X2: 24
NAVARA CD SE 4X4: 23
NP300 DC XE 2.3D MT 4WD: 23
WINGLE 5 G 2.2 4X2 ELITE: 23
POER 2.0L DELUXE 4X4: 22
SAIL NB 1.5L LS: 22
NAVARA NP300 DC SE 2.3D MT: 21
NP300 DC S 2.5 MT: 21
TUCSON TL 2.0 MT 4WD PLUS FL: 20
VENUE QX 1.6 MT PLUS: 20
NAVARA DC SE 2.3D MT 4WD: 20
T60 DX E6 4X4 MT 2024: 20
FRONTIER D/C 2,5

In [ ]:
def clean_modelo(modelo):
    """
    Clean the 'Modelo' string by applying the following transformations:
    1. Keep the first and second words (if they exist).
    2. Remove any subsequent word that:
       a. Begins with a number.
       b. Has only two characters.
    3. Do not keep more than 2 words in total.
    4. Join the remaining words back into a single string.

    """
     # Split the string into words
    words = modelo.split()

    # Keep the first two words (if they exist)
    base_words = words[:2]

    # Filter subsequent words (starting from the 3rd)
    # Only keep words with length > 2 and that do NOT start with a digit
    subsequent_words = [
        w for w in words[2:]
        if len(w) > 3 and not w[0].isdigit()
    ]

    # Combine the base words with the filtered subsequent words
    cleaned_words = base_words + subsequent_words
    
    # Truncate the list to the first four words
    cleaned_words = cleaned_words[:2]
    
    # Join them back into a string
    return ' '.join(cleaned_words)
    

# Convert the 'Modelo' column to strings and handle NaNs
df['Modelo_simplified'] = df['Modelo'].astype(str)
# Apply the cleaning function to the 'Modelo' column
df['Modelo_simplified'] = df['Modelo_simplified'].apply(clean_modelo)


modelos = df.value_counts('Modelo_simplified')
for modelo, count in modelos.items():
    print(f'{modelo}: {count}')

#df['merge_var'] = df['Marca'] + ' ' + df['Tipo de Producto'] + ' ' + df['Modelo']
df['merge_var'] = df['Marca'] + ' ' + df['Modelo_simplified']


NP300 DC: 216
H-1 TQ: 180
HILUX 2.4: 126
T60 DX: 121
HILUX 2.4L: 117
TUCSON NX4: 97
HILUX 4X4: 84
FRONTIER D/C: 79
VIGUS WORK: 76
NAVARA 2.3D: 73
T60 2.8L: 69
NAVARA NP300: 67
TUCSON TL: 61
POER 2.0L: 53
L200 4X4: 53
WINGLE 5: 52
SANTA FE: 50
HILUX DX: 43
NEW RAV4: 43
NAVARA CD: 42
T60 4X4: 41
PORTER 2.5: 40
BT-50 DOBLE: 37
NAVARA DC: 37
COLORADO LT: 36
CRETA GRAND: 33
NEW DMAX: 33
VIGUS DOBLE: 32
SAIL NB: 30
H-1 MB: 30
X-TRAIL SENSE: 29
RAV4 LUJO: 29
VENUE QX: 29
X-TRAIL 2.5L: 28
GRAND I-10: 28
L200 4X2: 28
HILUX 4X2: 26
ALL NEW: 25
NEW RANGER: 25
KICKS 1.6L: 24
WINGLE7 2.0L: 22
L200 KATANA: 22
DMAX CC: 21
D-MAX CC: 20
NEW D-MAX: 20
TRANSIT MINIBUS: 19
MUSSO GRAND: 19
700 1.4L: 19
L-200 D/C: 19
4RUNNER 4X4: 18
NEW MAZDA6: 18
AMAROK TRENDLINE: 18
TRAVELLER BUSINESS: 17
HUNTER 1.9L: 17
GRAND NOMADE: 17
HILUX 2.7: 16
TIGGO 2: 16
VERSA SENSE: 16
AZERA IG: 15
GRAND VITARA: 15
WIDE S-LONG: 15
HILUX SR: 15
VIGUS NEW: 15
FRONTIER P/UP: 15
NUEVO YARIS: 14
PICK UP: 14
T90 4X4: 14
MONTERO SPORT:

### 2. reduce number of models in web/scraped data

In [275]:
df_char = pd.read_csv("../6_car_analysis/csvs/scraped_data.csv")  

# Convert the 'Modelo' column to strings and handle NaNs
df_char['simplified_model'] = df_char['model'].astype(str)
# Apply the cleaning function to the 'Modelo' column
df_char['simplified_model'] = df_char['simplified_model'].apply(clean_modelo)


# --- 1. Remove unwanted words from `simplified_model` ---
words_to_remove = ['highline', 'ltz', 'xlt', 'xls', 'elite', 'plus', 'luxury', 'comfort', 'pick', 'pick-up', 'high', 'z71', 'diesel', 'sport', 'ltd', 'limited', '-up', 'premier', 'advance', ]
# Build a pattern that matches these words as whole words (case-insensitive)
pattern = r'\b(?:' + '|'.join(words_to_remove) + r')\b'

df_char['simplified_model'] = (
    df_char['simplified_model']
      # (optional) convert to lowercase to make matching easier
      .str.lower()  
      # remove each unwanted word if it appears as a separate word
      .str.replace(pattern, '', regex=True)
      # collapse extra spaces
      .str.replace(r'\s+', ' ', regex=True)
      # strip leading/trailing spaces
      .str.strip()
)

In [276]:
# --- 2. Group by the cleaned `simplified_model` and keep one row per group ---
# We will define a function to pick the "most common non-NaN" value in each column
def most_common_nonan(series):
    """Return the most frequent non-NaN value in `series`, or NaN if all are NaN."""
    s_no_nan = series.dropna()
    if len(s_no_nan) == 0:
        return np.nan
    else:
        # value_counts() sorts by freq desc by default; take the index of the top.
        return s_no_nan.value_counts().idxmax()

store_df = df_char.copy()

# Apply the aggregator to every column
# This will produce one row per unique `simplified_model`
df_char = (
    df_char
    .groupby('simplified_model', dropna=False)
    .agg(most_common_nonan)
    .reset_index()  # if you want `simplified_model` as a column instead of index
)

df_char['merge_var2'] = df_char['simplified_model']

In [277]:
import pandas as pd
from fuzzywuzzy import process, fuzz

# Prepare data for merge
df['merge_var'] = df['merge_var'].fillna('').astype(str).str.lower()
df_char['merge_var2'] = df_char['merge_var2'].fillna('').astype(str).str.lower()

def best_fuzzy_match(df1, df2, key1, key2, threshold):
    """
    For each row in df1, find the closest match in df2 based on a similarity score.
    
    Parameters:
    - df1: The DataFrame containing the base rows to match.
    - df2: The DataFrame containing the candidate rows to match.
    - key1: Column in df1 to use for matching.
    - key2: Column in df2 to use for matching.
    - threshold: Minimum similarity score to include a match.

    Returns:
    - DataFrame with matches and similarity scores.
    """
    matches = []
    for i, row in df1.iterrows():
        # Find the best match in df2 for the current row in df1
        best_match = process.extractOne(row[key1], df2[key2], scorer=fuzz.token_sort_ratio)
        if best_match and best_match[1] >= threshold:
            matches.append({
                key1: row[key1],  # Original value from df1
                key2: best_match[0],  # Best match from df2
                'similarity': best_match[1],  # Similarity score
                'row_id_df': row.name,  # Preserve df1 row index
                'row_id_df_char': df2[df2[key2] == best_match[0]].index[0]  # Match df2 row index
            })
    return pd.DataFrame(matches)

# Perform the fuzzy matching
threshold = 60  # Set your similarity threshold
result = best_fuzzy_match(df, df_char, key1='merge_var', key2='merge_var2', threshold=threshold)

# Merge the results back with the original DataFrames
final_result = result.merge(df, left_on='row_id_df', right_index=True, how='left') \
                     .merge(df_char, left_on='row_id_df_char', right_index=True, how='left', suffixes=('_df', '_df_char'))

# Display the final merged DataFrame
print(final_result.shape) #with .7 there are 3463 rows, with . 6 there are 3894 rows 


(3894, 157)


In [278]:
final_result.to_csv(os.path.join(output_path, 'transac_with_chars.csv'), index=False)